In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats as stats

In [ ]:
def load_csvs(folder):
    data_path = f"C:\\path\\to\\data\\{folder}"
    df = pd.read_csv(f"{data_path}\\super_res_test.csv", index_col=0, header=[0, 1])
    textures = pd.read_csv(f"{data_path}\\super_res_test_texture.csv", index_col=0)

    def _change_expt_name(expt):
        expt = folder.replace("_nores", "") + '_' +  expt
        return expt

    df.columns = pd.MultiIndex.from_tuples([(c[0], _change_expt_name(c[1])) for c in df.columns])
    textures.columns = [_change_expt_name(c) for c in textures.columns]

    return df, textures

In [ ]:
quality_deep, texture_deep = load_csvs("deep")
quality_shallow, texture_shallow = load_csvs("shallow")
quality_multi, texture_multi = load_csvs("multi")
quality_df = pd.concat([quality_deep, quality_shallow, quality_multi], axis=1)
texture_df = pd.concat([texture_deep, texture_shallow, texture_multi], axis=1)
texture_df.drop(labels=["shallow_LQ", "shallow_HQ", "multi_LQ", "multi_HQ"], axis=1, inplace=True)
texture_df.rename({"deep_LQ": "LQ", "deep_HQ": "HQ"}, axis=1, inplace=True)

In [ ]:
# expts = ["deep_sr_vq0", "shallow_sr_vq0", "multi_sr_vq0"]
# expts = ["deep_sr_vq0", "deep_sr_vq1", "deep_jo_vq64", "deep_jo_vq512"]
# expts = ["shallow_sr_vq0", "shallow_sr_vq128", "shallow_jo_vq8", "shallow_jo_vq256"]
expts = ["multi_sr_vq0", "multi_sr_vq128-128-256-512-512", "multi_jo_vq128-64-256-32-512", "multi_jo_vq128-128-512-512-1024"]

# quality_df.loc[:, pd.IndexSlice["MSE", expts]].quantile([0.05, 0.5, 0.95])
texture_df.loc[:, ["LQ", "HQ"] + expts].quantile([0.05, 0.5, 0.95])

In [ ]:
plt.figure(figsize=(18, 18))
plt.subplot(3, 1, 1)
plt.boxplot(quality_df["MSE"])
plt.xticks(ticks=np.arange(1, quality_df["MSE"].shape[1] + 1), labels=quality_df["MSE"].columns, rotation="vertical")
plt.subplot(3, 1, 2)
plt.boxplot(quality_df["pSNR"])
plt.xticks(ticks=np.arange(1, quality_df["pSNR"].shape[1] + 1), labels=quality_df["pSNR"].columns, rotation="vertical")
plt.subplot(3, 1, 3)
plt.boxplot(quality_df["SSIM"])
plt.xticks(ticks=np.arange(1, quality_df["SSIM"].shape[1] + 1), labels=quality_df["SSIM"].columns, rotation="vertical")
plt.tight_layout()
plt.show()

In [ ]:
print(f"Alpha = {0.05 / 12}")
# expts = ["deep_sr_vq0", "shallow_sr_vq0", "multi_sr_vq0"]
# expts = ["deep_sr_vq0", "deep_sr_vq1", "deep_jo_vq64", "deep_jo_vq512"]
# expts = ["shallow_sr_vq0", "shallow_sr_vq128", "shallow_jo_vq8", "shallow_jo_vq256"]
expts = ["multi_sr_vq0", "multi_sr_vq128-128-256-512-512", "multi_jo_vq128-64-256-32-512", "multi_jo_vq128-128-512-512-1024"]

print(f"MSE: {stats.kruskal(*quality_df.loc[:, pd.IndexSlice['MSE', expts]].values.T.tolist())}")
print(f"pSNR: {stats.kruskal(*quality_df.loc[:, pd.IndexSlice['pSNR', expts]].values.T.tolist())}")
print(f"SSIM: {stats.kruskal(*quality_df.loc[:, pd.IndexSlice['SSIM', expts]].values.T.tolist())}")

print("MSE")
print(f"MSE base vs vq-sr: {stats.mannwhitneyu(quality_df.loc[:, pd.IndexSlice['MSE', 'multi_sr_vq0']], quality_df.loc[:, pd.IndexSlice['MSE', 'multi_sr_vq128-128-256-512-512']])}")
print(f"MSE base vs jvq-sr: {stats.mannwhitneyu(quality_df.loc[:, pd.IndexSlice['MSE', 'multi_sr_vq0']], quality_df.loc[:, pd.IndexSlice['MSE', 'multi_jo_vq128-64-256-32-512']])}")
print(f"MSE base vs jvq-ce: {stats.mannwhitneyu(quality_df.loc[:, pd.IndexSlice['MSE', 'multi_sr_vq0']], quality_df.loc[:, pd.IndexSlice['MSE', 'multi_jo_vq128-128-512-512-1024']])}")
print(f"MSE vq-sr vs jvq-sr: {stats.mannwhitneyu(quality_df.loc[:, pd.IndexSlice['MSE', 'multi_sr_vq128-128-256-512-512']], quality_df.loc[:, pd.IndexSlice['MSE', 'multi_jo_vq128-64-256-32-512']])}")
print(f"MSE vq-sr vs jvq-ce: {stats.mannwhitneyu(quality_df.loc[:, pd.IndexSlice['MSE', 'multi_sr_vq128-128-256-512-512']], quality_df.loc[:, pd.IndexSlice['MSE', 'multi_jo_vq128-128-512-512-1024']])}")
print(f"MSE jvq-sr vs jvq-ce: {stats.mannwhitneyu(quality_df.loc[:, pd.IndexSlice['MSE', 'multi_jo_vq128-64-256-32-512']], quality_df.loc[:, pd.IndexSlice['MSE', 'multi_jo_vq128-128-512-512-1024']])}")

print("pSNR")
print(f"pSNR base vs vq-sr: {stats.mannwhitneyu(quality_df.loc[:, pd.IndexSlice['pSNR', 'multi_sr_vq0']], quality_df.loc[:, pd.IndexSlice['pSNR', 'multi_sr_vq128-128-256-512-512']])}")
print(f"pSNR base vs jvq-sr: {stats.mannwhitneyu(quality_df.loc[:, pd.IndexSlice['pSNR', 'multi_sr_vq0']], quality_df.loc[:, pd.IndexSlice['pSNR', 'multi_jo_vq128-64-256-32-512']])}")
print(f"pSNR base vs jvq-ce: {stats.mannwhitneyu(quality_df.loc[:, pd.IndexSlice['pSNR', 'multi_sr_vq0']], quality_df.loc[:, pd.IndexSlice['pSNR', 'multi_jo_vq128-128-512-512-1024']])}")
print(f"pSNR vq-sr vs jvq-sr: {stats.mannwhitneyu(quality_df.loc[:, pd.IndexSlice['pSNR', 'multi_sr_vq128-128-256-512-512']], quality_df.loc[:, pd.IndexSlice['pSNR', 'multi_jo_vq128-64-256-32-512']])}")
print(f"pSNR vq-sr vs jvq-ce: {stats.mannwhitneyu(quality_df.loc[:, pd.IndexSlice['pSNR', 'multi_sr_vq128-128-256-512-512']], quality_df.loc[:, pd.IndexSlice['pSNR', 'multi_jo_vq128-128-512-512-1024']])}")
print(f"pSNR jvq-sr vs jvq-ce: {stats.mannwhitneyu(quality_df.loc[:, pd.IndexSlice['pSNR', 'multi_jo_vq128-64-256-32-512']], quality_df.loc[:, pd.IndexSlice['pSNR', 'multi_jo_vq128-128-512-512-1024']])}")

In [ ]:
print(f"Alpha = {0.05 / 6}")

# expts = ["deep_sr_vq0", "shallow_sr_vq0", "multi_sr_vq0"]
# expts = ["deep_sr_vq0", "deep_sr_vq1", "deep_jo_vq64", "deep_jo_vq512"]
# expts = ["shallow_sr_vq0", "shallow_sr_vq128", "shallow_jo_vq8", "shallow_jo_vq256"]
expts = ["multi_sr_vq0", "multi_sr_vq128-128-256-512-512", "multi_jo_vq128-64-256-32-512", "multi_jo_vq128-128-512-512-1024"]

print(f"Texture: {stats.kruskal(*texture_df.loc[:, expts].values.T.tolist())}")
print(f"Texture base vs vq-sr: {stats.mannwhitneyu(texture_df.loc[:, 'multi_sr_vq0'], texture_df.loc[:, 'multi_sr_vq128-128-256-512-512'])}")
print(f"Texture base vs jvq-sr: {stats.mannwhitneyu(texture_df.loc[:, 'multi_sr_vq0'], texture_df.loc[:, 'multi_jo_vq128-64-256-32-512'])}")
print(f"Texture base vs jvq-ce: {stats.mannwhitneyu(texture_df.loc[:, 'multi_sr_vq0'], texture_df.loc[:, 'multi_jo_vq128-128-512-512-1024'])}")
print(f"Texture vq-sr vs jvq-sr: {stats.mannwhitneyu(texture_df.loc[:, 'multi_sr_vq128-128-256-512-512'], texture_df.loc[:, 'multi_jo_vq128-64-256-32-512'])}")
print(f"Texture vq-sr vs jvq-ce: {stats.mannwhitneyu(texture_df.loc[:, 'multi_sr_vq128-128-256-512-512'], texture_df.loc[:, 'multi_jo_vq128-128-512-512-1024'])}")
print(f"Texture jvq-sr vs jvq-ce: {stats.mannwhitneyu(texture_df.loc[:, 'multi_jo_vq128-64-256-32-512'], texture_df.loc[:, 'multi_jo_vq128-128-512-512-1024'])}")